In [ ]:

from itertools import combinations


transactions = [
    {'bread', 'milk', 'cheese'},
    {'bread', 'diapers', 'eggs', 'beer', 'cornflakes'},
    {'milk', 'diapers', 'eggs', 'cornflakes'},
    {'bread', 'milk', 'diapers', 'eggs'},
    {'bread', 'milk', 'cheese', 'diapers', 'eggs', 'cornflakes'}
]

min_support = 0.4
min_confidence = 0.6

def get_frequent_itemsets(transactions, k, min_support):
    itemsets = {}
    for transaction in transactions:
        for item in transaction:
            if item in itemsets:
                itemsets[item] += 1
            else:
                itemsets[item] = 1

    itemsets = {itemset: count for itemset, count in itemsets.items() if count >= min_support * len(transactions)}

    if k > 1:
        itemsets = {itemset: count for itemset, count in itemsets.items() if len(itemset) == k}
        candidates = list(itemsets.keys())
        itemsets = {}
        for transaction in transactions:
            for candidate in candidates:
                if set(candidate).issubset(transaction):
                    if candidate in itemsets:
                        itemsets[candidate] += 1
                    else:
                        itemsets[candidate] = 1
        itemsets = {itemset: count for itemset, count in itemsets.items() if count >= min_support * len(transactions)}
    return itemsets

def get_association_rules(frequent_itemsets, min_confidence):
    association_rules = {}
    for itemset, count in frequent_itemsets.items():
        if len(itemset) > 1:
            for i in range(1, len(itemset)):
                for antecedent in combinations(itemset, i):
                    antecedent = frozenset(antecedent)
                    consequent = itemset.difference(antecedent)
                    if antecedent in frequent_itemsets and consequent in frequent_itemsets:
                        confidence = count / frequent_itemsets[antecedent]
                        if confidence >= min_confidence:
                            association_rules[(antecedent, consequent)] = confidence
    return association_rules

frequent_itemsets = get_frequent_itemsets(transactions, 1, min_support)

k = 2
while frequent_itemsets:
    print(f'Frequent itemsets of length {k}:')
    print(frequent_itemsets)
    frequent_itemsets = get_frequent_itemsets(transactions, k, min_support)
    k += 1

association_rules = get_association_rules(frequent_itemsets, min_confidence)
print('Association rules:')
for rule, confidence in association_rules.items():
    antecedent, consequent = rule
    print(f'{antecedent} => {consequent}: {confidence}')

Frequent itemsets of length 2:
{'bread': 4, 'milk': 4, 'cheese': 2, 'cornflakes': 3, 'diapers': 4, 'eggs': 4}
Association rules:


In [ ]:
'''Suppose we have the following dataset that has various
transactions, and from this dataset, we need to find the frequent
itemsets and generate the association rules using the Apriori
algorithm:
'''
from itertools import chain, combinations
from collections import defaultdict

# Function to generate candidate itemsets
def candidate_itemsets(itemset, k):
    return set([i.union(j) for i in itemset for j in itemset if len(i.union(j)) == k])

# Function to generate frequent itemsets
def frequent_itemsets(transactions, min_support):
    itemset = set()
    for transaction in transactions:
        for item in transaction:
            itemset.add(frozenset([item]))

    # Generate frequent itemsets
    k = 1
    frequent_itemset = {}
    while True:
        if k > 1:
            itemset = candidate_itemsets(itemset, k)
        item_count = defaultdict(int)
        for transaction in transactions:
            for item in itemset:
                if item.issubset(transaction):
                    item_count[item] += 1
        new_frequent_itemset = {}
        for item, count in item_count.items():
            support = float(count) / len(transactions)
            if support >= min_support:
                new_frequent_itemset[item] = support
        if not new_frequent_itemset:
            break
        frequent_itemset.update(new_frequent_itemset)
        k += 1

    return frequent_itemset

# Function to generate association rules
def generate_rules(frequent_itemset, min_confidence):
    rules = []
    for item, support in frequent_itemset.items():
        if len(item) > 1:
            for element in item:
                left_side = item.difference(frozenset([element]))
                confidence = support / frequent_itemset[left_side]
                if confidence >= min_confidence:
                    rules.append((left_side, frozenset([element]), confidence))
    return rules

# Example dataset
transactions = [['A', 'B'], ['B', 'D'], ['B', 'C'], ['A', 'B', 'D'], ['A', 'C'], ['B', 'C'], ['A', 'C'], ['A', 'B', 'C', 'E'], ['A', 'B', 'C']]

# Find frequent itemsets with minimum support of 2
frequent_itemset = frequent_itemsets(transactions, 2/len(transactions))

# Generate association rules with minimum confidence of 50%
rules = generate_rules(frequent_itemset, 0.5)

# Print frequent itemsets
print("Frequent itemsets:")
for item, support in frequent_itemset.items():
    print("{} : {:.2f}".format(tuple(item), support))

# Print association rules
print("\nAssociation rules:")
for left, right, confidence in rules:
    print("{} => {} : {:.2f}".format(tuple(left), tuple(right), confidence))


Frequent itemsets:
('A',) : 0.67
('B',) : 0.78
('D',) : 0.22
('C',) : 0.67
('B', 'A') : 0.44
('B', 'D') : 0.22
('B', 'C') : 0.44
('A', 'C') : 0.44
('B', 'A', 'C') : 0.22

Association rules:
('A',) => ('B',) : 0.67
('B',) => ('A',) : 0.57
('D',) => ('B',) : 1.00
('C',) => ('B',) : 0.67
('B',) => ('C',) : 0.57
('C',) => ('A',) : 0.67
('A',) => ('C',) : 0.67
('A', 'C') => ('B',) : 0.50
('B', 'C') => ('A',) : 0.50
('B', 'A') => ('C',) : 0.50
